<a href="https://colab.research.google.com/github/bicicura/esports-data-miner/blob/main/Spanish_Lora_Trainer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ⭐ Entrenador de Lora de Hollowstrawberry

Basado en el trabajo de [Kohya_ss](https://github.com/kohya-ss/sd-scripts) y [Linaqruf](https://colab.research.google.com/github/Linaqruf/kohya-trainer/blob/main/kohya-LoRA-dreambooth.ipynb#scrollTo=-Z4w3lfFKLjr). ¡Gracias!

### ⭕ Disclaimer
The purpose of this document is to research bleeding-edge technologies in the field of machine learning.  
Please read and follow the [Google Colab guidelines](https://research.google.com/colaboratory/faq.html) and its [Terms of Service](https://research.google.com/colaboratory/tos_v3.html).

In [2]:
!nvidia-smi

| |GitHub|🇬🇧 English|🇪🇸 Spanish|
|:--|:-:|:-:|:-:|
| 🏠 **Origen** | [![GitHub](https://raw.githubusercontent.com/hollowstrawberry/kohya-colab/main/assets/github.svg)](https://github.com/hollowstrawberry/kohya-colab) | | |
| 📊 **Dataset Maker** | [![GitHub](https://raw.githubusercontent.com/hollowstrawberry/kohya-colab/main/assets/github.svg)](https://github.com/hollowstrawberry/kohya-colab/blob/main/Dataset_Maker.ipynb) | [![Open in Colab](https://raw.githubusercontent.com/hollowstrawberry/kohya-colab/main/assets/colab-badge.svg)](https://colab.research.google.com/github/hollowstrawberry/kohya-colab/blob/main/Dataset_Maker.ipynb) | [![Abrir en Colab](https://raw.githubusercontent.com/hollowstrawberry/kohya-colab/main/assets/colab-badge-spanish.svg)](https://colab.research.google.com/github/hollowstrawberry/kohya-colab/blob/main/Spanish_Dataset_Maker.ipynb) |
| ⭐ **Lora Trainer** | [![GitHub](https://raw.githubusercontent.com/hollowstrawberry/kohya-colab/main/assets/github.svg)](https://github.com/hollowstrawberry/kohya-colab/blob/main/Lora_Trainer.ipynb) | [![Open in Colab](https://raw.githubusercontent.com/hollowstrawberry/kohya-colab/main/assets/colab-badge.svg)](https://colab.research.google.com/github/hollowstrawberry/kohya-colab/blob/main/Lora_Trainer.ipynb) | [![Abrir en Colab](https://raw.githubusercontent.com/hollowstrawberry/kohya-colab/main/assets/colab-badge-spanish.svg)](https://colab.research.google.com/github/hollowstrawberry/kohya-colab/blob/main/Spanish_Lora_Trainer.ipynb) |
| 🌟 **XL Lora Trainer** | [![GitHub](https://raw.githubusercontent.com/hollowstrawberry/kohya-colab/main/assets/github.svg)](https://github.com/hollowstrawberry/kohya-colab/blob/main/Lora_Trainer_XL.ipynb) | [![Open in Colab](https://raw.githubusercontent.com/hollowstrawberry/kohya-colab/main/assets/colab-badge.svg)](https://colab.research.google.com/github/hollowstrawberry/kohya-colab/blob/main/Lora_Trainer_XL.ipynb) |  |
| 🌟 **Legacy XL Trainer** | [![GitHub](https://raw.githubusercontent.com/hollowstrawberry/kohya-colab/main/assets/github.svg)](https://github.com/hollowstrawberry/kohya-colab/blob/main/Lora_Trainer_XL_Legacy.ipynb) | [![Open in Colab](https://raw.githubusercontent.com/hollowstrawberry/kohya-colab/main/assets/colab-badge.svg)](https://colab.research.google.com/github/hollowstrawberry/kohya-colab/blob/main/Lora_Trainer_XL_Legacy.ipynb) |  |

In [ ]:
import os
import re
import toml
import shutil
import zipfile
from time import time
from IPython.display import Markdown, display

# These carry information from past executions
if "model_url" in globals():
  old_model_url = model_url
else:
  old_model_url = None
if "dependencies_installed" not in globals():
  dependencies_installed = False
if "model_file" not in globals():
  model_file = None

# These may be set by other cells, some are legacy
if "custom_dataset" not in globals():
  custom_dataset = None
if "override_dataset_config_file" not in globals():
  override_dataset_config_file = None
if "override_config_file" not in globals():
  override_config_file = None
if "optimizer" not in globals():
  optimizer = "AdamW"
if "optimizer_args" not in globals():
  optimizer_args = None
if "continue_from_lora" not in globals():
  continue_from_lora = ""
if "weighted_captions" not in globals():
  weighted_captions = False
if "adjust_tags" not in globals():
  adjust_tags = False
if "keep_tokens_weight" not in globals():
  keep_tokens_weight = 1.0

COLAB = True # low ram
XFORMERS = False
SOURCE = "https://github.com/kohya-ss/sd-scripts"
COMMIT = None
BETTER_EPOCH_NAMES = True
LOAD_TRUNCATED_IMAGES = True

#@title ## 🚩 Empieza Aquí

#@markdown ### ▶️ Base
#@markdown El nombre de tu proyecto también es el nombre de la carpeta donde irán tus imágenes. No se permiten espacios.
nombre_proyecto = "bicicura" #@param {type:"string"}
project_name = nombre_proyecto.strip()
#@markdown La estructura de carpetas no importa y es por comodidad. Asegúrate de siempre elegir la misma. Me gusta organizar por proyecto.
estructura_de_carpetas = "Organizar por proyecto (MyDrive/Loras/nombre_proyecto/dataset)" #@param ["Organizar por categoría (MyDrive/lora_training/datasets/nombre_proyecto)", "Organizar por proyecto (MyDrive/Loras/nombre_proyecto/dataset)"]
folder_structure = estructura_de_carpetas
#@markdown Decidir el modelo base de entrenamiento. Los modelos por defecto producen los resultados más limpios y consistentes. Puedes cambiarlo por un modelo propio si lo deseas.
modelo_de_entrenamiento = "Stable Diffusion (sd-v1-5-pruned-noema-fp16.safetensors)" #@param ["Anime (animefull-final-pruned-fp16.safetensors)", "AnyLora (AnyLoRA_noVae_fp16-pruned.ckpt)", "Stable Diffusion (sd-v1-5-pruned-noema-fp16.safetensors)"]
opcional_enlace_a_modelo_propio = "" #@param {type:"string"}
modelo_propio_basado_en_sd2 = False #@param {type:"boolean"}
custom_model_is_based_on_sd2 = modelo_propio_basado_en_sd2

if opcional_enlace_a_modelo_propio:
  model_url = opcional_enlace_a_modelo_propio
elif "AnyLora" in modelo_de_entrenamiento:
  model_url = "https://huggingface.co/Lykon/AnyLoRA/resolve/main/AnyLoRA_noVae_fp16-pruned.ckpt"
elif "Anime" in modelo_de_entrenamiento:
  model_url = "https://huggingface.co/hollowstrawberry/stable-diffusion-guide/resolve/main/models/animefull-final-pruned-fp16.safetensors"
else:
  model_url = "https://huggingface.co/hollowstrawberry/stable-diffusion-guide/resolve/main/models/sd-v1-5-pruned-noema-fp16.safetensors"

#@markdown ### ▶️ Procesamiento <p>
#@markdown La resolución de 512 es estándar en Stable Diffusion 1.5. No es necesario recortar o achicar, el proceso es automático.
resolucion = 512 #@param {type:"slider", min:512, max:1024, step:128}
resolution = resolucion
#@markdown Esta opción va a voltear tus imágenes para así tener el doble y aprender mejor. <p>
#@markdown **Desactiva esto si te importan los elementos asimétricos en tu Lora.**
flip_aug = False #@param {type:"boolean"}
#markdown Leave empty for no captions.
caption_extension = "" #param {type:"string"}
#@markdown Mezclar las tags de anime ayuda al aprendizaje. Una tag de activación va al inicio de cada archivo de texto y no se mezclará.
mezclar_tags = False #@param {type:"boolean"}
shuffle_caption = mezclar_tags
tags_de_activacion = "0" #@param [0,1,2,3]
keep_tokens = int(tags_de_activacion)

#@markdown ### ▶️ Pasos <p>
#@markdown Tus imágenes se repetirán este número de veces durante el entrenamiento. Recomiendo que el valor total sea entre 200 y 400.
num_repeats = 6 #@param {type:"number"}
#@markdown Cuánto tiempo deseas entrenar. Un buen punto de partida puede ser alrededor de 10 epochs o alrededor de 2000 pasos. <p>
#@markdown Un epoch es una cantidad de pasos igual a: tu cantidad de imágenes multipliccada por sus repeticiones, y dividido en el batch size.
unidad_preferida = "Epochs" #@param ["Epochs", "Pasos"]
cuantos = 7 #@param {type:"number"}
max_train_epochs = cuantos if unidad_preferida == "Epochs" else None
max_train_steps = cuantos if unidad_preferida == "Pasos" else None
#@markdown Guardar más epochs te permitirá comparar mejor el progreso de tu Lora.
guardar_cada_cuantos_epochs = 1 #@param {type:"number"}
save_every_n_epochs = guardar_cada_cuantos_epochs
guardar_solo_ultimos_epochs = 10 #@param {type:"number"}
keep_only_last_n_epochs = guardar_solo_ultimos_epochs
if not save_every_n_epochs:
  save_every_n_epochs = max_train_epochs
if not keep_only_last_n_epochs:
  keep_only_last_n_epochs = max_train_epochs
#@markdown Un batch size mayor hace el entrenamiento más rápido, pero puede empeorar el aprendizaje. Se recomienda 2 o 3.
batch_size = 1 #@param {type:"slider", min:1, max:8, step:1}
train_batch_size = batch_size

#@markdown ### ▶️ Aprendizaje
#@markdown La tasa de aprendizaje es lo más importante. Si deseas entrenar más lento con muchas imágenes, o si tienes un dim y alpha altos, usa un unet de 2e-4 o menor. <p>
#@markdown El text encoder ayuda a tu Lora a aprender conceptos un poco mejor. Se recomienda la mitad o un quinto del unet. Puedes dejarlo en 0 para algunos estilos.
aprendizaje_unet = 2e-4 #@param {type:"number"}
unet_lr = aprendizaje_unet
aprendizaje_text_encoder = 1e-5 #@param {type:"number"}
text_encoder_lr = aprendizaje_text_encoder
#@markdown El scheduler es el algoritmo matemático que guiará el entrenamiento. Para personajes recomiendo `cosine_with_restarts` con un valor de 3. Si no estás seguro ponlo en `constant` e ignora el valor.
scheduler = "cosine_with_restarts" #@param ["constant", "cosine", "cosine_with_restarts", "constant_with_warmup", "linear", "polynomial"]
lr_scheduler = scheduler
valor_de_scheduler = 3 #@param {type:"number"}
lr_scheduler_number = valor_de_scheduler
lr_scheduler_num_cycles = lr_scheduler_number if lr_scheduler == "cosine_with_restarts" else 0
lr_scheduler_power = lr_scheduler_number if lr_scheduler == "polynomial" else 0
#@markdown Pasos de calentamiento durante el entrenamiento para un inicio eficiente. Recomiendo dejarlo en 5%.
calentamiento = 0.05 #@param {type:"slider", min:0.0, max:0.5, step:0.01}
lr_warmup_ratio = calentamiento
lr_warmup_steps = 0
#@markdown Nueva función que hace el aprendizaje mucho más eficiente. Puede que tus Loras estén listos en la mitad de epochs. Se usará un valor de 5.0 como en la [investigación](https://arxiv.org/abs/2303.09556).
min_snr_gamma = True #@param {type:"boolean"}
min_snr_gamma_value = 5.0 if min_snr_gamma else None

#@markdown ### ▶️ Estructura
#@markdown LoRA es el clásico y útil para muchos usos. LoCon es bueno con estilos ya que aprende con más capas.
lora_type = "LoRA" #@param ["LoRA", "LoCon"]

#@markdown Aquí hay algunos valores recomendados para las opciones de abajo:

#@markdown | type | network_dim | network_alpha | conv_dim | conv_alpha |
#@markdown | :---: | :---: | :---: | :---: | :---: |
#@markdown | LoRA | 16 | 8 |   |   |
#@markdown | LoCon | 16 | 8 | 8 | 4 |

#@markdown Un dim mayor equivale a un Lora más grande, pero no siempre es mejor. Se recomienda de 8 a 32, con un alpha igual a la mitad del dim.
network_dim = 16 #@param {type:"slider", min:1, max:128, step:1}
network_alpha = 8 #@param {type:"slider", min:1, max:128, step:1}
#@markdown Los siguientes sólo aplican a las capas adicionales de LoCon.
conv_dim = 8 #@param {type:"slider", min:1, max:64, step:1}
conv_alpha = 4 #@param {type:"slider", min:1, max:64, step:1}

network_module = "networks.lora"
network_args = None
if lora_type.lower() == "locon":
  network_args = [f"conv_dim={conv_dim}", f"conv_alpha={conv_alpha}"]

#@markdown ### ▶️ Listo
#@markdown Ahora puedes correr esta celda apretando el botón circular a la izquierda. ¡Buena suerte!


# 👩‍💻 Cool code goes here

if optimizer.lower() == "prodigy" or "dadapt" in optimizer.lower():
  if override_values_for_dadapt_and_prodigy:
    unet_lr = 0.5
    text_encoder_lr = 0.5
    lr_scheduler = "constant_with_warmup"
    lr_warmup_ratio = 0.05
    network_alpha = network_dim

  if not optimizer_args:
    optimizer_args = ["decouple=True","weight_decay=0.01","betas=[0.9,0.999]"]
    if optimizer == "Prodigy":
      optimizer_args.extend(["d_coef=2","use_bias_correction=True"])
      if lr_warmup_ratio > 0:
        optimizer_args.append("safeguard_warmup=True")
      else:
        optimizer_args.append("safeguard_warmup=False")

root_dir = "/content" if COLAB else "~/Loras"
deps_dir = os.path.join(root_dir, "deps")
repo_dir = os.path.join(root_dir, "kohya-trainer")

if "/Loras" in folder_structure:
  main_dir      = os.path.join(root_dir, "drive/MyDrive/Loras") if COLAB else root_dir
  log_folder    = os.path.join(main_dir, "_logs")
  config_folder = os.path.join(main_dir, project_name)
  images_folder = os.path.join(main_dir, project_name, "dataset")
  output_folder = os.path.join(main_dir, project_name, "output")
else:
  main_dir      = os.path.join(root_dir, "drive/MyDrive/lora_training") if COLAB else root_dir
  images_folder = os.path.join(main_dir, "datasets", project_name)
  output_folder = os.path.join(main_dir, "output", project_name)
  config_folder = os.path.join(main_dir, "config", project_name)
  log_folder    = os.path.join(main_dir, "log")

config_file = os.path.join(config_folder, "training_config.toml")
dataset_config_file = os.path.join(config_folder, "dataset_config.toml")
accelerate_config_file = os.path.join(repo_dir, "accelerate_config/config.yaml")

def install_dependencies():
  os.chdir(root_dir)

  # ✅ Re-clonar limpio (evita líos de git pull + sed patches)
  if os.path.exists(repo_dir):
    !rm -rf "{repo_dir}"
  !git clone {SOURCE} "{repo_dir}"

  os.chdir(repo_dir)

  if COMMIT:
    !git reset --hard "{COMMIT}"

  !wget https://raw.githubusercontent.com/hollowstrawberry/kohya-colab/main/train_network_wrapper.py -q -O train_network_wrapper.py
  !wget https://raw.githubusercontent.com/hollowstrawberry/kohya-colab/main/dracula.py -q -O dracula.py

  !apt -y update -qq
  !apt -y install aria2 -qq

  # ✅ Fix numpy/opencv (lo más importante para cv2)
  !pip -q install "numpy<2"
  !pip -q uninstall -y opencv-python opencv-contrib-python opencv-python-headless
  !pip -q install "opencv-python-headless==4.8.1.78"

  # ✅ Dependencias del trainer
  !pip -q install accelerate==0.25.0 transformers==4.36.2 diffusers[torch]==0.25.0 ftfy==6.1.1 \
    einops==0.7.0 pytorch-lightning==1.9.0 \
    bitsandbytes==0.43.0 \
    prodigyopt==1.0 lion-pytorch==0.0.6 tensorboard safetensors==0.4.2 altair==4.2.2 \
    easygui==0.98.3 toml==0.10.2 voluptuous==0.13.1 huggingface-hub==0.20.1 imagesize==1.4.1 \
    rich==13.7.1 torch==2.4.1 triton

  !pip -q install -e .

  if XFORMERS:
    !pip -q install xformers==0.0.28.post1

  # patches (robusto a distintos repos)
  if COLAB:
    # algunos forks tenían este archivo; si no existe, no pasa nada
    if os.path.exists("library/model_util.py"):
      !sed -i "s@cpu@cuda@" library/model_util.py

  if LOAD_TRUNCATED_IMAGES:
    from pathlib import Path

    candidates = [
      Path("library/train_util.py"),
      Path("train_util.py"),
      Path("library/utils.py"),
    ]

    target = next((p for p in candidates if p.exists()), None)

    if target is None:
      print("No encontré train_util.py (ni similares). Skip patch LOAD_TRUNCATED_IMAGES.")
    else:
      s = target.read_text()
      needle = "from PIL import Image"
      insert = "from PIL import Image, ImageFile\nImageFile.LOAD_TRUNCATED_IMAGES=True"

      if "ImageFile.LOAD_TRUNCATED_IMAGES" not in s and needle in s:
        s = s.replace(needle, insert, 1)
        target.write_text(s)
        print(f"Patched {target} OK")
      else:
        print(f"Patch ya aplicado o no encontré '{needle}' en {target}.")

  if BETTER_EPOCH_NAMES:
    if os.path.exists("library/train_util.py"):
      !sed -i 's/{:06d}/{:02d}/g' library/train_util.py
    if os.path.exists("train_network.py"):
      !sed -i 's/"." + args.save_model_as)/"-{:02d}.".format(num_train_epochs) + args.save_model_as)/g' train_network.py

    from accelerate.utils import write_basic_config
    if not os.path.exists(accelerate_config_file):
      write_basic_config(save_location=accelerate_config_file)

    os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
    os.environ["BITSANDBYTES_NOWELCOME"] = "1"
    os.environ["SAFETENSORS_FAST_GPU"] = "1"

def validate_dataset():
  global lr_warmup_steps, lr_warmup_ratio, caption_extension, keep_tokens, keep_tokens_weight, weighted_captions, adjust_tags
  supported_types = (".png", ".jpg", ".jpeg", ".webp", ".bmp")

  print("\n💿 Revisando archivos...")
  if not project_name.strip() or any(c in project_name for c in " .()\"'\\/"):
    print("💥 Error: Por favor elije un nombre de proyecto válido.")
    return

  if custom_dataset:
    try:
      datconf = toml.loads(custom_dataset)
      datasets = [d for d in datconf["datasets"][0]["subsets"]]
    except:
      print(f"💥 Error: ¡Tu configuración de datos propia es inválida o tiene errores! Por favor revisa el ejemplo original.")
      return
    reg = [d.get("image_dir") for d in datasets if d.get("is_reg", False)]
    datasets_dict = {d["image_dir"]: d["num_repeats"] for d in datasets}
    folders = datasets_dict.keys()
    files = [f for folder in folders for f in os.listdir(folder)]
    images_repeats = {folder: (len([f for f in os.listdir(folder) if f.lower().endswith(supported_types)]), datasets_dict[folder]) for folder in folders}
  else:
    reg = []
    folders = [images_folder]
    files = os.listdir(images_folder)
    images_repeats = {images_folder: (len([f for f in files if f.lower().endswith(supported_types)]), num_repeats)}

  for folder in folders:
    if not os.path.exists(folder):
      print(f"💥 Error: La carpeta {folder.replace('/content/drive/', '')} no existe.")
      return
  for folder, (img, rep) in images_repeats.items():
    if not img:
      print(f"💥 Error: La carpeta {folder.replace('/content/drive/', '')} está vacía.")
      return
  for f in files:
    if not f.lower().endswith((".txt", ".npz")) and not f.lower().endswith(supported_types):
      print(f"💥 Error: Archivo inválido encontrado: \"{f}\". Abortando.")
      return

  if not [txt for txt in files if txt.lower().endswith(".txt")]:
    caption_extension = ""
  if continue_from_lora and not (continue_from_lora.endswith(".safetensors") and os.path.exists(continue_from_lora)):
    print(f"💥 Error: Archivo de continuar_lora inválido. Ejemplo: /content/drive/MyDrive/Loras/ejemplo.safetensors")
    return

  pre_steps_per_epoch = sum(img*rep for (img, rep) in images_repeats.values())
  steps_per_epoch = pre_steps_per_epoch/train_batch_size
  total_steps = max_train_steps or int(max_train_epochs*steps_per_epoch)
  estimated_epochs = int(total_steps/steps_per_epoch)
  lr_warmup_steps = int(total_steps*lr_warmup_ratio)

  for folder, (img, rep) in images_repeats.items():
    print("📁"+folder.replace("/content/drive/", "") + (" (Regularización)" if folder in reg else ""))
  print(f"📈 Se encontraron {img} imágenes con {rep} repeticiones, equivalente a {img*rep} pasos.")
  print(f"📉 Divide {pre_steps_per_epoch} pasos en {train_batch_size} batch size para obtener {steps_per_epoch} pasos por epoch.")
  if max_train_epochs:
    print(f"🔮 Habrá {max_train_epochs} epochs, para un total de alrededor de {total_steps} pasos totales.")
  else:
    print(f"🔮 Habrá {total_steps} pasos, divididos en {estimated_epochs} epochs y un poco más.")

  if total_steps > 10000:
    print("💥 Error: Tus pasos totales on muy altos. Probablemente cometiste un error. Abortando...")
    return

  if adjust_tags:
    print(f"\n📎 Weighted tags: {'ON' if weighted_captions else 'OFF'}")
    if weighted_captions:
      print(f"📎 Will use {keep_tokens_weight} weight on {keep_tokens} activation tag(s)")
    print("📎 Adjusting tags...")
    adjust_weighted_tags(folders, keep_tokens, keep_tokens_weight, weighted_captions)

  return True

def adjust_weighted_tags(folders, keep_tokens: int, keep_tokens_weight: float, weighted_captions: bool):
  weighted_tag = re.compile(r"\((.+?):[.\d]+\)(,|$)")
  for folder in folders:
    for txt in [f for f in os.listdir(folder) if f.lower().endswith(".txt")]:
      with open(os.path.join(folder, txt), 'r') as f:
        content = f.read()
      # reset previous changes
      content = content.replace('\\', '')
      content = weighted_tag.sub(r'\1\2', content)
      if weighted_captions:
        # re-apply changes
        content = content.replace(r'(', r'\(').replace(r')', r'\)').replace(r':', r'\:')
        if keep_tokens_weight > 1:
          tags = [s.strip() for s in content.split(",")]
          for i in range(min(keep_tokens, len(tags))):
            tags[i] = f'({tags[i]}:{keep_tokens_weight})'
          content = ", ".join(tags)
      with open(os.path.join(folder, txt), 'w') as f:
        f.write(content)

def create_config():
  global dataset_config_file, config_file, model_file

  if override_config_file:
    config_file = override_config_file
    print(f"\n⭕ Usando configuración propia {config_file}")
  else:
    config_dict = {
      "additional_network_arguments": {
        "unet_lr": unet_lr,
        "text_encoder_lr": text_encoder_lr,
        "network_dim": network_dim,
        "network_alpha": network_alpha,
        "network_module": network_module,
        "network_args": network_args,
        "network_train_unet_only": True if text_encoder_lr == 0 else None,
        "network_weights": continue_from_lora if continue_from_lora else None
      },
      "optimizer_arguments": {
        "learning_rate": unet_lr,
        "lr_scheduler": lr_scheduler,
        "lr_scheduler_num_cycles": lr_scheduler_num_cycles if lr_scheduler == "cosine_with_restarts" else None,
        "lr_scheduler_power": lr_scheduler_power if lr_scheduler == "polynomial" else None,
        "lr_warmup_steps": lr_warmup_steps if lr_scheduler != "constant" else None,
        "optimizer_type": optimizer,
        "optimizer_args": optimizer_args if optimizer_args else None,
      },
      "training_arguments": {
        "max_train_steps": max_train_steps,
        "max_train_epochs": max_train_epochs,
        "save_every_n_epochs": save_every_n_epochs,
        "save_last_n_epochs": keep_only_last_n_epochs,
        "train_batch_size": train_batch_size,
        "noise_offset": None,
        "clip_skip": 1,
        "min_snr_gamma": min_snr_gamma_value,
        "weighted_captions": weighted_captions,
        "seed": 42,
        "max_token_length": 225,
        "xformers": XFORMERS,
        "lowram": COLAB,
        "max_data_loader_n_workers": 8,
        "persistent_data_loader_workers": True,
        "save_precision": "fp16",
        "mixed_precision": "fp16",
        "output_dir": output_folder,
        "logging_dir": log_folder,
        "output_name": project_name,
        "log_prefix": project_name,
      },
      "model_arguments": {
        "pretrained_model_name_or_path": model_file,
        "v2": custom_model_is_based_on_sd2,
        "v_parameterization": True if custom_model_is_based_on_sd2 else None,
      },
      "saving_arguments": {
        "save_model_as": "safetensors",
      },
      "dreambooth_arguments": {
        "prior_loss_weight": 1.0,
      },
      "dataset_arguments": {
        "cache_latents": False,
      },
    }

    for key in config_dict:
      if isinstance(config_dict[key], dict):
        config_dict[key] = {k: v for k, v in config_dict[key].items() if v is not None}

    with open(config_file, "w") as f:
      f.write(toml.dumps(config_dict))
    print(f"\n📄 Configuración guardada en {config_file}")

  if override_dataset_config_file:
    dataset_config_file = override_dataset_config_file
    print(f"⭕ Usando configuración de datos propia {dataset_config_file}")
  else:
    dataset_config_dict = {
      "general": {
        "resolution": resolution,
        "shuffle_caption": shuffle_caption,
        "keep_tokens": keep_tokens,
        "flip_aug": flip_aug,
        "caption_extension": caption_extension,
        "enable_bucket": True,
        "bucket_reso_steps": 64,
        "bucket_no_upscale": False,
        "min_bucket_reso": 320 if resolution > 640 else 256,
        "max_bucket_reso": 1280 if resolution > 640 else 1024,
      },
      "datasets": toml.loads(custom_dataset)["datasets"] if custom_dataset else [
        {
          "subsets": [
            {
              "num_repeats": num_repeats,
              "image_dir": images_folder,
              "class_tokens": None if caption_extension else project_name
            }
          ]
        }
      ]
    }

    for key in dataset_config_dict:
      if isinstance(dataset_config_dict[key], dict):
        dataset_config_dict[key] = {k: v for k, v in dataset_config_dict[key].items() if v is not None}

    with open(dataset_config_file, "w") as f:
      f.write(toml.dumps(dataset_config_dict))
    print(f"📄 Configuración de datos guardada en {dataset_config_file}")

def download_model():
  global old_model_url, model_url, model_file
  real_model_url = model_url.strip()

  if real_model_url.lower().endswith((".ckpt", ".safetensors")):
    model_file = f"/content{real_model_url[real_model_url.rfind('/'):]}"
  else:
    model_file = "/content/downloaded_model.safetensors"
    if os.path.exists(model_file):
      !rm "{model_file}"

  if m := re.search(r"(?:https?://)?(?:www\.)?huggingface\.co/[^/]+/[^/]+/blob", model_url):
    real_model_url = real_model_url.replace("blob", "resolve")
  elif m := re.search(r"(?:https?://)?(?:www\.)?civitai\.com/models/([0-9]+)", model_url):
    real_model_url = f"https://civitai.com/api/download/models/{m.group(1)}"

  !aria2c "{real_model_url}" --console-log-level=warn -c -s 16 -x 16 -k 10M -d / -o "{model_file}"

  if model_file.lower().endswith(".safetensors"):
    from safetensors.torch import load_file as load_safetensors
    try:
      test = load_safetensors(model_file)
      del test
    except:
      #if "HeaderTooLarge" in str(e):
      new_model_file = os.path.splitext(model_file)[0]+".ckpt"
      !mv "{model_file}" "{new_model_file}"
      model_file = new_model_file
      print(f"Modelo renombrado a {os.path.splitext(model_file)[0]}.ckpt")

  if model_file.lower().endswith(".ckpt"):
    from torch import load as load_ckpt
    try:
      test = load_ckpt(model_file)
      del test
    except:
      return False

  return True

def main():
  global dependencies_installed

  if COLAB and not os.path.exists('/content/drive'):
    from google.colab import drive
    print("📂 Conectando a Google Drive...")
    drive.mount('/content/drive')

  for dir in (main_dir, deps_dir, repo_dir, log_folder, images_folder, output_folder, config_folder):
    os.makedirs(dir, exist_ok=True)

  if not validate_dataset():
    return

  if not dependencies_installed:
    print("\n🏭 Instalando...\n")
    t0 = time()
    install_dependencies()
    t1 = time()
    dependencies_installed = True
    print(f"\n✅ Instalación completada en {int(t1-t0)} segundos.")
  else:
    print("\n✅ Ya se ha realizado la instalación.")

  if old_model_url != model_url or not model_file or not os.path.exists(model_file):
    print("\n🔄 Descargando modelo...")
    if not download_model():
      print("\n💥 Error: El modelo que elegiste es inválido o está corrupto, o no se pudo encontrar. Recomiendo usar un enlace de huggingface o civitai.")
      return
    print()
  else:
    print("\n🔄 El modelo ya ha sido descargado.\n")

  create_config()

  print("\n⭐ Iniciando entrenador...\n")
  os.chdir(repo_dir)

  !accelerate launch --config_file={accelerate_config_file} --num_cpu_threads_per_process=1 train_network_wrapper.py --dataset_config={dataset_config_file} --config_file={config_file}

main()



💿 Revisando archivos...
📁MyDrive/Loras/bicicura/dataset
📈 Se encontraron 49 imágenes con 6 repeticiones, equivalente a 294 pasos.
📉 Divide 294 pasos en 1 batch size para obtener 294.0 pasos por epoch.
🔮 Habrá 7 epochs, para un total de alrededor de 2058 pasos totales.

🏭 Instalando...

Cloning into '/content/kohya-trainer'...
remote: Enumerating objects: 10785, done.
remote: Counting objects: 100% (104/104), done.
remote: Compressing objects: 100% (71/71), done.
remote: Total 10785 (delta 56), reused 41 (delta 33), pack-reused 10681 (from 2)
Receiving objects: 100% (10785/10785), 12.66 MiB | 6.25 MiB/s, done.
Resolving deltas: 100% (7707/7707), done.
95 packages can be upgraded. Run 'apt list --upgradable' to see them.
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
aria2 is already the newest version (1.36.0-1).
0 upgraded, 0 newly installed, 0 

/usr/local/lib/python3.12/dist-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.12/dist-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(



✅ Instalación completada en 32 segundos.

🔄 Descargando modelo...

Download Results:
gid   |stat|avg speed  |path/URI
======+====+===========+=======================================================
c1b96b|OK  |       0B/s|//content/sd-v1-5-pruned-noema-fp16.safetensors

Status Legend:
(OK):download completed.


📄 Configuración guardada en /content/drive/MyDrive/Loras/bicicura/training_config.toml
📄 Configuración de datos guardada en /content/drive/MyDrive/Loras/bicicura/dataset_config.toml

⭐ Iniciando entrenador...

/usr/local/lib/python3.12/dist-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.12/dist-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytr